In [6]:
import BSE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import itertools
import random

sess_id = "q3_demo_default"

start_time = 0
end_time = 60 * 10

# trader population: basic background traders + 1 MMM01
buyers_spec = [('ZIP', 5), ('ZIC', 5)]
sellers_spec = [('ZIP', 5), ('SHVR', 5)]
mrktmakers_spec = [
    ('MMM01', 1, {})
]

traders_spec = {
    'buyers': buyers_spec,
    'sellers': sellers_spec,
    'mrktmakers': mrktmakers_spec
}

sup_range = (100, 200)
dem_range = (100, 200)

supply_schedule = [
    {'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}
]
demand_schedule = [
    {'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}
]

order_schedule = {
    'sup': supply_schedule,
    'dem': demand_schedule,
    'interval': 30,
    'timemode': 'drip-jitter'
}

dumpfile_flags = {
    'dump_strats': False,
    'dump_lobs': False,
    'dump_tape': False,
    'dump_blotters': False,
    'dump_avgbals': True
}

BSE.market_session(
    sess_id=sess_id,
    starttime=start_time,
    endtime=end_time,
    trader_spec=traders_spec,
    order_schedule=order_schedule,
    dumpfile_flags=dumpfile_flags,
    sess_vrbs=False,
)

print(f"Session {sess_id} finished. Should create {sess_id}_avg_balance.csv")


MM01 init: n_past_trades=1, bid_percent=0.50000, ask_delta=25

Session q3_demo_default finished. Should create q3_demo_default_avg_balance.csv
Demo MMM01 profit (approx): 0.0


In [7]:
def get_mmm01_profit_from_avg_balance(sess_id, initial_balance=500.0):
    """
    read <sess_id>_avg_balance.csv and compute MMM01's final avg balance,
    then subtract the initial balance to get an approximate profit.
    """
    fname = f"{sess_id}_avg_balance.csv"
    if not os.path.exists(fname):
        raise FileNotFoundError(f"{fname} not found. Did you enable dump_avgbals and run the session?")

    with open(fname, 'r') as f:
        rows = list(csv.reader(f))

    last_row = rows[-1]

    mmm_avg_balance = None

    for i in range(4, len(last_row), 4):
        ttype = last_row[i].strip()
        if ttype == 'MMM01':
            try:
                avg_balance = float(last_row[i + 3])
            except (ValueError, IndexError):
                continue
            mmm_avg_balance = avg_balance

    if mmm_avg_balance is None:
        raise ValueError("No MMM01 data found in avg_balance file.")

    profit = mmm_avg_balance - initial_balance
    return profit

demo_profit = get_mmm01_profit_from_avg_balance("q3_demo_default")
print("Demo MMM01 profit (approx):", demo_profit)

Demo MMM01 profit (approx): 0.0
